# 1.1 First NGSolve example

Let us solve the Poisson problem of finding $u$ satisfying 

$$
\begin{aligned}
-\Delta u & = f && \text { in  the unit square},
\\
u & = 0 && \text{ on the bottom and right parts of the boundary},
\\
\frac{\partial u }{\partial n } & = 0 
&& \text{ on the remaining  boundary parts}.
\end{aligned}
$$

## Quick steps to solution:

#### 1. Import NGSolve and Netgen Python modules:

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import unit_square

importing NGSolve-6.2.2204


#### 2. Generate an unstructured mesh

In [2]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.01))
mesh.nv, mesh.ne   # number of vertices & elements 
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

BaseWebGuiScene

In [14]:
help(unit_square)

Help on SplineGeometry in module netgen.libngpy._geom2d object:

class SplineGeometry(netgen.libngpy._meshing.NetgenGeometry)
 |  a 2d boundary representation geometry model by lines and splines
 |  
 |  Method resolution order:
 |      SplineGeometry
 |      netgen.libngpy._meshing.NetgenGeometry
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  AddCircle lambda geo, c, r, **args
 |  
 |  AddCurve(...)
 |      AddCurve(self: netgen.libngpy._geom2d.SplineGeometry, func: object, leftdomain: int = 1, rightdomain: int = 0, bc: object = <netgen.libngpy._meshing.NGDummyArgument object at 0x7fb540109530>, maxh: float = 1e+99) -> None
 |      
 |      Curve is given as parametrization on the interval [0,1]
 |  
 |  AddPoint lambda *args, **kwargs
 |  
 |  AddRectangle lambda geo, p1, p2, **args
 |  
 |  AddSegment lambda *args, **kwargs
 |  
 |  Append(...)
 |      Append(self: netgen.libngpy._geom2d.SplineGeometry, point_indices: list, 

In [13]:
help(mesh)

Help on Mesh in module ngsolve.comp object:

class Mesh(pybind11_builtins.pybind11_object)
 |  NGSolve interface to the Netgen mesh. Provides access and functionality
 |  to use the mesh for finite element calculations.
 |  
 |  Parameters:
 |  
 |  mesh (netgen.Mesh): a mesh generated from Netgen
 |  
 |  Method resolution order:
 |      Mesh
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  BBBoundaries(...)
 |      BBBoundaries(self: ngsolve.comp.Mesh, pattern: str) -> ngsolve.comp.Region
 |      
 |      Return co dim 3 boundary mesh-region matching the given regex pattern
 |  
 |  BBoundaries(...)
 |      BBoundaries(self: ngsolve.comp.Mesh, pattern: str) -> ngsolve.comp.Region
 |      
 |      Return co dim 2 boundary mesh-region matching the given regex pattern
 |  
 |  Boundaries(...)
 |      Boundaries(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. Boundaries(self: ngsolve.comp.Mesh, pattern: str) -> ng

* Here we prescribed a maximal mesh-size of 0.2 using the `maxh` flag. 


#### 3. Declare a finite element space:

In [3]:
fes = H1(mesh, order=1, dirichlet="bottom|right|left|top")
fes.ndof  # number of unknowns in this space

11786

Python's help system displays further documentation.

In [4]:
help(fes)

Help on H1 in module ngsolve.comp object:

class H1(FESpace)
 |  An H1-conforming finite element space.
 |  
 |  The H1 finite element space consists of continuous and
 |  element-wise polynomial functions. It uses a hierarchical (=modal)
 |  basis built from integrated Legendre polynomials on tensor-product elements,
 |  and Jaboci polynomials on simplicial elements. 
 |  
 |  Boundary values are well defined. The function can be used directly on the
 |  boundary, using the trace operator is optional.
 |  
 |  The H1 space supports variable order, which can be set individually for edges, 
 |  faces and cells. 
 |  
 |  Internal degrees of freedom are declared as local dofs and are eliminated 
 |  if static condensation is on.
 |  
 |  The wirebasket consists of all vertex dofs. Optionally, one can include the 
 |  first (the quadratic bubble) edge basis function, or all edge basis functions
 |  into the wirebasket.
 |  
 |  Keyword arguments can be:
 |  
 |  order: int = 1
 |    order

#### 4. Declare test function, trial function, and grid function 

* Test and trial function are symbolic objects - called `ProxyFunctions` -  that help you construct bilinear forms (and have no space to hold solutions). 

* `GridFunctions`, on the other hand, represent functions in the finite element space and contains memory to hold coefficient vectors.

In [5]:
u = fes.TrialFunction()  # symbolic object
v = fes.TestFunction()   # symbolic object
gfu = GridFunction(fes)  # solution 

Alternately, you can get both the trial and test variables at once:

In [6]:
u, v = fes.TnT()

#### 5. Define and assemble linear and bilinear forms:

In [7]:
a = BilinearForm(fes, symmetric=True)
a += grad(u)*grad(v)*dx
a.Assemble()

f = LinearForm(fes)
f += x*v*dx
f.Assemble()

You can examine the linear system in more detail:

In [8]:
print(f.vec)

 4.16667e-08
 2.66766e-05
 1.6625e-05
 8.75905e-08
 5.07896e-07
 1.02303e-06
 1.48838e-06
 1.92864e-06
 2.32598e-06
 2.70381e-06
 3.09331e-06
 3.49772e-06
 3.91904e-06
 4.34504e-06
 4.77456e-06
 5.20468e-06
 5.63493e-06
 6.06588e-06
 6.49701e-06
 6.92902e-06
 7.36137e-06
 7.79414e-06
 8.22704e-06
 8.66001e-06
 9.093e-06
 9.526e-06
 9.959e-06
 1.0392e-05
 1.0825e-05
 1.1258e-05
 1.1691e-05
 1.2124e-05
 1.2557e-05
 1.299e-05
 1.3423e-05
 1.3856e-05
 1.4289e-05
 1.4722e-05
 1.5155e-05
 1.5588e-05
 1.6021e-05
 1.6454e-05
 1.6887e-05
 1.732e-05
 1.7753e-05
 1.8186e-05
 1.8619e-05
 1.9052e-05
 1.9485e-05
 1.9918e-05
 2.0351e-05
 2.0784e-05
 2.1217e-05
 2.165e-05
 2.2083e-05
 2.2516e-05
 2.2949e-05
 2.3382e-05
 2.3815e-05
 2.4248e-05
 2.4681e-05
 2.5114e-05
 2.5547e-05
 2.598e-05
 2.6413e-05
 2.6846e-05
 2.7279e-05
 2.7712e-05
 2.8145e-05
 2.8578e-05
 2.9011e-05
 2.9444e-05
 2.9877e-05
 3.031e-05
 3.0743e-05
 3.1176e-05
 3.1609e-05
 3.2042e-05
 3.2475e-05
 3.2908e-05
 3.3341e-05
 3.3774e-05
 

In [9]:
print(a.mat)

Row 0:   0: 1   4: -0.5   399: -0.5
Row 1:   1: 0.851359   102: -0.305471   103: -0.300529   498: -0.24536
Row 2:   2: 1   201: -0.5   202: -0.5
Row 3:   3: 0.853059   300: -0.309012   301: -0.304453   695: -0.239593
Row 4:   0: -0.5   4: 1.8793   5: -0.395554   399: -0.191765   400: -0.791979
Row 5:   4: -0.395554   5: 1.76232   6: -0.37017   400: -0.405557   401: -0.591039
Row 6:   5: -0.37017   6: 1.74715   7: -0.361844   401: -0.485844   402: -0.529294
Row 7:   6: -0.361844   7: 1.74221   8: -0.340512   402: -0.503168   403: -0.536688
Row 8:   7: -0.340512   8: 1.73674   9: -0.320015   403: -0.528582   404: -0.547634
Row 9:   8: -0.320015   9: 1.73364   10: -0.304519   404: -0.551037   405: -0.558066
Row 10:   9: -0.304519   10: 1.73247   11: -0.297018   405: -0.567974   406: -0.56296
Row 11:   10: -0.297018   11: 1.73217   12: -0.292962   406: -0.573658   407: -0.568538
Row 12:   11: -0.292962   12: 1.73209   13: -0.29153   407: -0.575583   408: -0.572014
Row 13:   12: -0.29153   

#### 6. Solve the system:

In [15]:
gfu.vec.data = \
    a.mat.Inverse(freedofs=fes.FreeDofs(),inverse="sparsecholesky") * f.vec
Draw(gfu)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

NameError: name 'plt' is not defined

The Dirichlet boundary condition constrains some degrees of freedom. The argument `fes.FreeDofs()` indicates that only the remaining "free" degrees of freedom should participate in the linear solve.

You can examine the coefficient vector of solution if needed:

In [11]:
print(gfu.vec)

       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
 

## Ways to interact with NGSolve

* A jupyter notebook (like this one) gives you one way to interact with NGSolve. When you have a complex sequence of tasks to perform, the notebook may not be adequate.


* You can write an entire python module in a text editor and call python on the command line. (A script of the above is provided in `poisson.py`.)
    ```
    python3 poisson.py
    ```
  
* If you want the Netgen GUI, then use `netgen` on the command line:
    ```
    netgen poisson.py
    ```
  You can then ask for a python shell from the GUI's menu options (`Solve -> Python shell`).
  